In [1]:
import pandas as pd

In [7]:
df = pd.read_csv("../Datasets/jobs_dataset_with_features.csv")

In [8]:
df.shape

(1615940, 2)

In [9]:
# # Dropping classes with less than 6500 instances
# min_count = 6500
# role_counts = df['Role'].value_counts()
# dropped_classes = role_counts[role_counts < min_count].index
# filtered_df = df[~df['Role'].isin(dropped_classes)].reset_index(drop=True)

# # Checking the updated role counts
# filtered_df['Role'].value_counts()

In [10]:
#len(filtered_df['Role'].value_counts())
len(df['Role'].value_counts())

376

In [11]:
#df = filtered_df.sample(n=10000)
df = df.sample(n=10000)

In [12]:
df.head(300)

,Role,Features
1208595,Interaction Designer,4 to 9 Years UX/UI Designer BA Interaction des...
1340263,In-House Counsel,5 to 13 Years Legal Advisor B.Tech In-house co...
1120994,Urban Planner,0 to 9 Years Landscape Architect BBA Urban des...
1412432,Intellectual Property Lawyer,3 to 14 Years Legal Counsel M.Tech Intellectua...
206876,NoSQL Database Engineer,5 to 13 Years Database Developer MCA NoSQL dat...
...,...,...
618016,Clinical Psychologist,1 to 11 Years Psychologist BCA Clinical psycho...
1286958,Subject Matter Expert,4 to 12 Years Teacher BBA Expertise in a speci...
1060855,Quality Control Manager,1 to 10 Years Operations Manager M.Tech Qualit...
1048686,HVAC Systems Designer,1 to 9 Years Mechanical Designer B.Tech HVAC s...


# TFIDF

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
# Splitting the data into features (X) and target (y)
X = df['Features']
y = df['Role']
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [14]:
# RandomForestClassifier
#rf_classifier = RandomForestClassifier()
#clf = svm.SVC()
reg = LogisticRegression()
#rf_classifier.fit(X_train_tfidf, y_train)
reg.fit(X_train_tfidf, y_train)
#clf.fit(X_train_tfidf, y_train)
# Predictions
#y_pred = rf_classifier.predict(X_test_tfidf)
y_pred =reg.predict(X_test_tfidf)
#y_pred =clf.predict(X_test_tfidf)
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test,y_pred))

Accuracy: 1.0
                                          precision    recall  f1-score   support

                           API Developer       1.00      1.00      1.00         2
                 Accessibility Developer       1.00      1.00      1.00         1
                       Account Executive       1.00      1.00      1.00         7
                         Account Manager       1.00      1.00      1.00         6
                      Account Strategist       1.00      1.00      1.00         3
                   Accounting Controller       1.00      1.00      1.00         8
                      Accounting Manager       1.00      1.00      1.00         6
           Acute Care Nurse Practitioner       1.00      1.00      1.00         6
                     Addiction Counselor       1.00      1.00      1.00         6
                Administrative Assistant       1.00      1.00      1.00         5
              Administrative Coordinator       1.00      1.00      1.00         6
 

# Recommendation

In [15]:
# Clean resume
import re
def cleanResume(txt):
    cleanText = re.sub('http\S+\s', ' ', txt)
    cleanText = re.sub('RT|cc', ' ', cleanText)
    cleanText = re.sub('#\S+\s', ' ', cleanText)
    cleanText = re.sub('@\S+', '  ', cleanText)  
    cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText) 
    cleanText = re.sub('\s+', ' ', cleanText)
    return cleanText


# Prediction and Category Name
def job_recommendation(resume_text):
    resume_text= cleanResume(resume_text)
    resume_tfidf = tfidf_vectorizer.transform([resume_text])
    #predicted_category = rf_classifier.predict(resume_tfidf)[0]
    predicted_category = reg.predict(resume_tfidf)[0]
    #predicted_category = clf.predict(resume_tfidf)[0]
    return predicted_category

In [16]:
# Example Usage
resume_file = """Objective:
A creative and detail-oriented Designer with a passion for visual communication and brand identity seeking opportunities to leverage design skills in a dynamic and collaborative environment.

Education:
- Bachelor of Fine Arts in Graphic Design, XYZ College, GPA: 3.7/4.0
- Diploma in Web Design, ABC Institute, GPA: 3.9/4.0

Skills:
- Proficient in Adobe Creative Suite (Photoshop, Illustrator, InDesign)
- Strong understanding of typography, layout, and color theory
- Experience in both print and digital design
- Ability to conceptualize and execute design projects from concept to completion
- Excellent attention to detail and time management skills

Experience:
Graphic Designer | XYZ Design Studio
- Created visually appealing graphics for various marketing materials, including brochures, flyers, and social media posts
- Collaborated with clients to understand their design needs and deliver creative solutions that align with their brand identity
- Worked closely with the marketing team to ensure consistency in brand messaging across all platforms

Freelance Designer
- Designed logos, branding materials, and website layouts for small businesses and startups
- Managed multiple projects simultaneously while meeting tight deadlines and maintaining quality standards
- Established and maintained strong client relationships through clear communication and exceptional service

Projects:
- Rebranding Campaign for XYZ Company: Led a team to redesign the company's logo, website, and marketing collateral, resulting in a 30% increase in brand recognition
- Packaging Design for ABC Product Launch: Developed eye-catching packaging designs for a new product line, contributing to a successful launch and positive customer feedback

Certifications:
- Adobe Certified Expert (ACE) in Adobe Illustrator
- Responsive Web Design Certification from Udemy

Languages:
- English (Native)
- Spanish (Intermediate)
"""
predicted_category = job_recommendation(resume_file)
print("Predicted Category:", predicted_category)

Predicted Category: User Interface Designer


In [17]:
# Example Usage
resume_file = """Objective:
Dedicated and results-oriented Banking professional with a strong background in financial analysis and customer service seeking opportunities to contribute to a reputable financial institution. Eager to leverage expertise in risk management, investment strategies, and relationship building to drive business growth and client satisfaction.

Education:
- Bachelor of Business Administration in Finance, XYZ University, GPA: 3.8/4.0
- Certified Financial Analyst (CFA) Level I Candidate

Skills:
- Proficient in financial modeling and analysis using Excel, Bloomberg Terminal, and other financial software
- Extensive knowledge of banking products and services, including loans, mortgages, and investment products
- Strong understanding of regulatory compliance and risk management practices in the banking industry
- Excellent communication and interpersonal skills, with a focus on building rapport with clients and colleagues
- Ability to work efficiently under pressure and adapt to changing market conditions

Experience:
Financial Analyst | ABC Bank
- Conducted financial analysis and risk assessment for corporate clients, including credit analysis, financial statement analysis, and cash flow modeling
- Developed customized financial solutions to meet clients' needs and objectives, resulting in increased revenue and client retention
- Collaborated with cross-functional teams to identify new business opportunities and optimize existing processes

Customer Service Representative | DEF Bank
- Provided exceptional customer service to bank clients, addressing inquiries, resolving issues, and promoting banking products and services
- Processed transactions accurately and efficiently, including deposits, withdrawals, and account transfers
- Educated customers on various banking products and services, helping them make informed financial decisions

Internship | GHI Investments
- Assisted portfolio managers with investment research and analysis, including industry and company-specific research, financial modeling, and performance analysis
- Prepared investment presentations and reports for clients, highlighting investment opportunities and performance metrics
- Conducted market research and analysis to identify trends and opportunities in the financial markets

Certifications:
- Certified Financial Planner (CFP)
- Series 7 and Series 63 Securities Licenses

Languages:
- English (Native)
- Spanish (Proficient)

"""
predicted_category = job_recommendation(resume_file)
print("Predicted Category:", predicted_category)

Predicted Category: Financial Analyst


In [18]:
import pickle
#pickle.dump(rf_classifier,open('rf_classifier_job_recommendation.pkl','wb'))
pickle.dump(reg,open('rf_classifier_job_recommendation.pkl','wb'))
#pickle.dump(clf,open('rf_classifier_job_recommendation.pkl','wb'))
pickle.dump(tfidf_vectorizer,open('tfidf_vectorizer_job_recommendation.pkl','wb'))